## 감독 코드 추출

In [1]:
import requests
import json

director_name = input()
director_list_url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleList.json?key='+키+'&peopleNm='+director_name
res = requests.get(director_list_url)
text = res.text

d = json.loads(text)
print(d)

정이삭
{'peopleListResult': {'totCnt': 2, 'peopleList': [{'peopleCd': '20288012', 'peopleNm': '정이삭', 'peopleNmEn': '', 'repRoleNm': '배우', 'filmoNames': '파이프라인|세자매|양자물리학|조작된 도시'}, {'peopleCd': '10062125', 'peopleNm': '정이삭', 'peopleNmEn': 'Lee Isaac CHUNG', 'repRoleNm': '감독', 'filmoNames': '미나리|문유랑가보'}], 'source': '영화진흥위원회'}}


In [2]:
director_list = []
filmo_list = []

for director in d['peopleListResult']['peopleList']:
  if director['repRoleNm'] == '감독':
    filmo_list = director['filmoNames'].split('|')
    director_list.append([director['peopleCd'],director['peopleNm'],director['peopleNmEn'],
                     director['repRoleNm'],filmo_list])
  

print(director_list[0][0])
print(director_list)

10062125
[['10062125', '정이삭', 'Lee Isaac CHUNG', '감독', ['미나리', '문유랑가보']]]


In [3]:
import pandas as pd

director_df = pd.DataFrame(director_list)
director_df.columns = ['peopleCd','peopleNm','peopleNmEn','repRoleNm','filmoNames']
# director_df
print(director_df['filmoNames'].iloc[0][0].__class__)
print(director_df['filmoNames'].iloc[0][:5])
print(director_df['filmoNames'].values.flatten())
# director_df['filmoNames'] = director_df['filmoNames'].astype(str)
# print(director_df['filmoNames'].head())
print(director_df['peopleCd'][0])

<class 'str'>
['미나리', '문유랑가보']
[list(['미나리', '문유랑가보'])]
10062125


## 영화 코드 추출

In [4]:
movieCd_list = []
for director in director_df['peopleCd']:
  movie_url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/people/searchPeopleInfo.json?key='+키+'&peopleCd='+director
  res = requests.get(movie_url)
  text = res.text
  d = json.loads(text)
  for movie in d['peopleInfoResult']['peopleInfo']['filmos']:
    if movie['moviePartNm'] == '감독':
      movieCd_list.append([movie['movieCd'], movie['movieNm']])
  print(d)
  print(movieCd_list)

{'peopleInfoResult': {'peopleInfo': {'peopleCd': '10062125', 'peopleNm': '정이삭', 'peopleNmEn': 'Lee Isaac CHUNG', 'sex': '남자', 'repRoleNm': '감독', 'homepages': [], 'filmos': [{'movieCd': '20205144', 'movieNm': '미나리', 'moviePartNm': '감독'}, {'movieCd': '20078788', 'movieNm': '문유랑가보', 'moviePartNm': '감독'}]}, 'source': '영화진흥위원회'}}
[['20205144', '미나리'], ['20078788', '문유랑가보']]


In [5]:
movieCd_df = pd.DataFrame(movieCd_list)
movieCd_df.columns = ['movieCd', 'movieNm']
movieCd_df

,movieCd,movieNm
0,20205144,미나리
1,20078788,문유랑가보


## 평점 추출

In [6]:
import os
import sys
import urllib.request
from urllib.parse import quote
import json
import re
import requests

In [7]:
client_id = your_id
client_secret = your_pw

In [8]:
film_list = []
for movieNm in movieCd_df['movieNm']:
  encText = urllib.parse.quote(movieNm)
  url = "https://openapi.naver.com/v1/search/movie.json?query=" + encText + "&display=100" # json 결과
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()
  if(rescode==200):
    response_body = json.load(response)
    film_list.append(response_body)
    print(response_body)
  else:
    print("Error Code:" + rescode)

print(film_list)

{'lastBuildDate': 'Fri, 20 Aug 2021 14:49:23 +0900', 'total': 2, 'start': 1, 'display': 2, 'items': [{'title': '<b>미나리</b>', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187310', 'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1873/187310_P20_100054.jpg', 'subtitle': 'Minari', 'pubDate': '2020', 'director': '정이삭|', 'actor': '윌 패튼|스티븐 연|한예리|윤여정|앨런 김|노엘 조|', 'userRating': '7.63'}, {'title': '고수풀과 <b>미나리</b>', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=114274', 'image': '', 'subtitle': 'Cilantro y perejil', 'pubDate': '1998', 'director': '라파엘 몬테로|', 'actor': '아셀리아 라미레즈|데미안 비쉬어|', 'userRating': '0.00'}]}
{'lastBuildDate': 'Fri, 20 Aug 2021 14:49:23 +0900', 'total': 1, 'start': 1, 'display': 1, 'items': [{'title': '무뉴랑가보', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=67815', 'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0678/67815_P13_194719.jpg', 'subtitle': 'Munyurangabo', 'pubDate': '2007', 'director': '정이삭|', 'actor': '', 'use

In [9]:
print(type(film_list[0]))

<class 'dict'>


In [10]:
film_df = pd.DataFrame(film_list)
film_df

,lastBuildDate,total,start,display,items
0,"Fri, 20 Aug 2021 14:49:23 +0900",2,1,2,"[{'title': '<b>미나리</b>', 'link': 'https://movi..."
1,"Fri, 20 Aug 2021 14:49:23 +0900",1,1,1,"[{'title': '무뉴랑가보', 'link': 'https://movie.nav..."


In [11]:
film_df.astype({'display': int})

,lastBuildDate,total,start,display,items
0,"Fri, 20 Aug 2021 14:49:23 +0900",2,1,2,"[{'title': '<b>미나리</b>', 'link': 'https://movi..."
1,"Fri, 20 Aug 2021 14:49:23 +0900",1,1,1,"[{'title': '무뉴랑가보', 'link': 'https://movie.nav..."


In [12]:
rating_list = []

for i in range(film_df.shape[0]):
  for j in range(film_df['display'][i]):
    if director_name in film_df['items'][i][j]['director']:
      rating_list.append([film_df['items'][i][j]['title'].replace("<b>","").replace("</b>",""),film_df['items'][i][j]['link'],film_df['items'][i][j]['image'],
                          film_df['items'][i][j]['subtitle'].replace("<b>","").replace("</b>",""),
                          film_df['items'][i][j]['pubDate'],film_df['items'][i][j]['director'],film_df['items'][i][j]['actor'],film_df['items'][i][j]['userRating']])

rating_list

[['미나리',
  'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187310',
  'https://ssl.pstatic.net/imgmovie/mdi/mit110/1873/187310_P20_100054.jpg',
  'Minari',
  '2020',
  '정이삭|',
  '윌 패튼|스티븐 연|한예리|윤여정|앨런 김|노엘 조|',
  '7.63'],
 ['무뉴랑가보',
  'https://movie.naver.com/movie/bi/mi/basic.nhn?code=67815',
  'https://ssl.pstatic.net/imgmovie/mdi/mit110/0678/67815_P13_194719.jpg',
  'Munyurangabo',
  '2007',
  '정이삭|',
  '',
  '9.00']]

In [13]:
rating_df = pd.DataFrame(rating_list)
rating_df.columns=['movieNm','link','image','submovieNm','pubDate','director','actor','userRating']
rating_df = rating_df.drop_duplicates(['movieNm'], keep='first', ignore_index=True)
rating_df_sorted = rating_df.sort_values(by=['userRating'], axis=0, ascending=False)

In [14]:
rating_df

,movieNm,link,image,submovieNm,pubDate,director,actor,userRating
0,미나리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/18...,Minari,2020,정이삭|,윌 패튼|스티븐 연|한예리|윤여정|앨런 김|노엘 조|,7.63
1,무뉴랑가보,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/06...,Munyurangabo,2007,정이삭|,,9.00


In [15]:
rating_df_sorted

,movieNm,link,image,submovieNm,pubDate,director,actor,userRating
1,무뉴랑가보,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/06...,Munyurangabo,2007,정이삭|,,9.00
0,미나리,https://movie.naver.com/movie/bi/mi/basic.nhn?...,https://ssl.pstatic.net/imgmovie/mdi/mit110/18...,Minari,2020,정이삭|,윌 패튼|스티븐 연|한예리|윤여정|앨런 김|노엘 조|,7.63


In [16]:
final_recommendation = rating_df_sorted.head(5)
final_recommendation
final_list = final_recommendation.values.tolist()
print(final_list)
print(len(final_list))

[['무뉴랑가보', 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=67815', 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0678/67815_P13_194719.jpg', 'Munyurangabo', '2007', '정이삭|', '', '9.00'], ['미나리', 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187310', 'https://ssl.pstatic.net/imgmovie/mdi/mit110/1873/187310_P20_100054.jpg', 'Minari', '2020', '정이삭|', '윌 패튼|스티븐 연|한예리|윤여정|앨런 김|노엘 조|', '7.63']]
2
